In [52]:
import pandas as pd
import sys
from os import listdir
from os.path import isfile, join
import lightgbm as lgb
import numpy as np
from tqdm import tqdm_notebook as tqdm
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from math import sqrt

In [2]:
lgb.__version__

'2.2.4'

In [3]:
is_kaggle_notebook = False
test_folder = './data/test/'
train_data_path = './data/train.csv'
submission_file_path = "./data/sample_submission.csv"
if is_kaggle_notebook == True:
    test_folder = '../input/test/'
    train_data_path = '../input/train.csv'
    submission_file_path = "../input/sample_submission.csv"

In [4]:
def genTrainFeatures(df,num_segments,segment_size):
    train_data = pd.DataFrame()
    
    for segment in tqdm(range(num_segments)):
        seg_start = segment*segment_size
        seg_end = segment*segment_size + segment_size
        
        mini_train_data = df[seg_start:seg_end]
        fft_val = pd.Series(np.fft.fft(mini_train_data.signal.values))
        
        window_sizes = [10, 50, 100, 1000]
        for window in window_sizes:
            rolling_window_values = mini_train_data.signal.rolling(window=window).mean()
            train_data.loc[segment,"rolling_mean_" + str(window)] = rolling_window_values.mean()
            train_data.loc[segment,"rolling_std_" + str(window)] = rolling_window_values.std()
            
            rolling_fft_window_values = fft_val.rolling(window=window).mean()
            train_data.loc[segment,"rolling_fft_mean_" + str(window)] = rolling_fft_window_values.mean()
            train_data.loc[segment,"rolling_fft_std_" + str(window)] = rolling_fft_window_values.std()
            
        q_25 = mini_train_data.signal.rolling(window=50).quantile(0.25)
        train_data.loc[segment,"rolling_q25_mean"] = q_25.mean()
        train_data.loc[segment,"rolling_q25_std"] = q_25.std()
        
        q_75 = mini_train_data.signal.rolling(window=50).quantile(0.75)
        train_data.loc[segment,"rolling_q75_mean"] = q_75.mean()
        train_data.loc[segment,"rolling_q75_std"] = q_75.std()
        
        q_50 = mini_train_data.signal.rolling(window=50).quantile(0.5)
        train_data.loc[segment,"rolling_q50_mean"] = q_50.mean()
        train_data.loc[segment,"rolling_q50_std"] = q_50.std()
        
        rolling_min = mini_train_data.signal.rolling(window=50).min()
        train_data.loc[segment,"rolling_min_mean"] = rolling_min.mean()
        train_data.loc[segment,"rolling_min_std"] = rolling_min.std()
        
        rolling_max = mini_train_data.signal.rolling(window=50).max()
        train_data.loc[segment,"rolling_max_mean"] = rolling_max.mean()
        train_data.loc[segment,"rolling_max_std"] = rolling_max.std()
        
        rolling_skew = mini_train_data.signal.rolling(window=50).skew()
        train_data.loc[segment,"rolling_skew_mean"] = rolling_skew.mean()
        train_data.loc[segment,"rolling_skew_std"] = rolling_skew.std()
        
        rolling_kurt = mini_train_data.signal.rolling(window=50).kurt()
        train_data.loc[segment,"rolling_kurt_mean"] = rolling_kurt.mean()
        train_data.loc[segment,"rolling_kurt_std"] = rolling_kurt.std()
        
        train_data.loc[segment,"rolling_iqr_mean"] = q_75.mean() - q_25.mean()
        train_data.loc[segment,"rolling_iqr_std"] = q_75.std() - q_25.std()
        
                    
        q_25_fft = fft_val.rolling(window=50).quantile(0.25)
        train_data.loc[segment,"rolling_fft_q25_mean"] = q_25_fft.mean()
        train_data.loc[segment,"rolling_fft_q25_std"] = q_25_fft.std()
        
        q_75_fft = fft_val.rolling(window=50).quantile(0.75)
        train_data.loc[segment,"rolling_fft_q75_mean"] = q_75_fft.mean()
        train_data.loc[segment,"rolling_fft_q75_std"] = q_75_fft.std()
        
        q_50_fft = fft_val.rolling(window=50).quantile(0.5)
        train_data.loc[segment,"rolling_fft_q50_mean"] = q_50_fft.mean()
        train_data.loc[segment,"rolling_fft_q50_std"] = q_50_fft.std()
        
        rolling_min_fft = fft_val.rolling(window=50).min()
        train_data.loc[segment,"rolling_fft_min_mean"] = rolling_min_fft.mean()
        train_data.loc[segment,"rolling_fft_min_std"] = rolling_min_fft.std()
        
        rolling_max_fft = fft_val.rolling(window=50).max()
        train_data.loc[segment,"rolling_fft_max_mean"] = rolling_max_fft.mean()
        train_data.loc[segment,"rolling_fft_max_std"] = rolling_max_fft.std()
        
        rolling_skew_fft = fft_val.rolling(window=50).skew()
        train_data.loc[segment,"rolling_fft_skew_mean"] = rolling_skew_fft.mean()
        train_data.loc[segment,"rolling_fft_skew_std"] = rolling_skew_fft.std()
        
        rolling_kurt_fft = fft_val.rolling(window=50).kurt()
        train_data.loc[segment,"rolling_fft_kurt_mean"] = rolling_kurt_fft.mean()
        train_data.loc[segment,"rolling_fft_kurt_std"] = rolling_kurt_fft.std()
        
        train_data.loc[segment,"rolling_fft_iqr_mean"] = q_75_fft.mean() - q_25_fft.mean()
        train_data.loc[segment,"rolling_fft_iqr_std"] = q_75_fft.std() - q_25_fft.std()
        
        quaketimes = mini_train_data.quaketime.values
        train_data.loc[segment,'quaketime'] = quaketimes[-1]
    
    return train_data

In [24]:
def genTestFeatures(tx):
    fft_val = pd.Series(np.fft.fft(tx))
    
    df = pd.DataFrame()
    window_sizes = [10, 50, 100, 1000]
    for window in window_sizes:
        rolling_window_values = tx.rolling(window=window).mean()
        df.loc[0,"rolling_mean_" + str(window)] = rolling_window_values.mean()
        df.loc[0,"rolling_std_" + str(window)] = rolling_window_values.std()
        
        rolling_fft_window_values = fft_val.rolling(window=window).mean()
        df.loc[0,"rolling_fft_mean_" + str(window)] = rolling_fft_window_values.mean()
        df.loc[0,"rolling_fft_std_" + str(window)] = rolling_fft_window_values.std()
    
    q_25 = tx.rolling(window=50).quantile(0.25)
    df.loc[0,"rolling_q25_mean"] = q_25.mean()
    df.loc[0,"rolling_q25_std"] = q_25.std()
    
    q_75 = tx.rolling(window=50).quantile(0.75)
    df.loc[0,"rolling_q75_mean"] = q_75.mean()
    df.loc[0,"rolling_q75_std"] = q_75.std()
    
    q_50 = tx.rolling(window=50).quantile(0.5)
    df.loc[0,"rolling_q50_mean"] = q_50.mean()
    df.loc[0,"rolling_q50_std"] = q_50.std()
    
    rolling_min = tx.rolling(window=50).min()
    df.loc[0,"rolling_min_mean"] = rolling_min.mean()
    df.loc[0,"rolling_min_std"] = rolling_min.std()
    
    rolling_max = tx.rolling(window=50).max()
    df.loc[0,"rolling_max_mean"] = rolling_max.mean()
    df.loc[0,"rolling_max_std"] = rolling_max.std()
    
    skew_values = tx.rolling(window=50).quantile(0.5).skew()
    df.loc[0,"rolling_skew_mean"] = skew_values.mean()
    df.loc[0,"rolling_skew_std"] = skew_values.std()
    
    kurt_values = tx.rolling(window=50).quantile(0.5).kurt()
    df.loc[0,"rolling_kurt_mean"] = kurt_values.mean()
    df.loc[0,"rolling_kurt_std"] = kurt_values.std()
    
    df.loc[0,"rolling_iqr_mean"] = q_75.mean() - q_25.mean()
    df.loc[0,"rolling_iqr_std"] = q_75.std() - q_25.std()
    
    
    q_25_fft = fft_val.rolling(window=50).quantile(0.25)
    df.loc[0,"rolling_fft_q25_mean"] = q_25_fft.mean()
    df.loc[0,"rolling_fft_q25_std"] = q_25_fft.std()

    q_75_fft = fft_val.rolling(window=50).quantile(0.75)
    df.loc[0,"rolling_fft_q75_mean"] = q_75_fft.mean()
    df.loc[0,"rolling_fft_q75_std"] = q_75_fft.std()

    q_50_fft = fft_val.rolling(window=50).quantile(0.5)
    df.loc[0,"rolling_fft_q50_mean"] = q_50_fft.mean()
    df.loc[0,"rolling_fft_q50_std"] = q_50_fft.std()

    rolling_min_fft = fft_val.rolling(window=50).min()
    df.loc[0,"rolling_fft_min_mean"] = rolling_min_fft.mean()
    df.loc[0,"rolling_fft_min_std"] = rolling_min_fft.std()

    rolling_max_fft = fft_val.rolling(window=50).max()
    df.loc[0,"rolling_fft_max_mean"] = rolling_max_fft.mean()
    df.loc[0,"rolling_fft_max_std"] = rolling_max_fft.std()

    rolling_skew_fft = fft_val.rolling(window=50).skew()
    df.loc[0,"rolling_fft_skew_mean"] = rolling_skew_fft.mean()
    df.loc[0,"rolling_fft_skew_std"] = rolling_skew_fft.std()

    rolling_kurt_fft = fft_val.rolling(window=50).kurt()
    df.loc[0,"rolling_fft_kurt_mean"] = rolling_kurt_fft.mean()
    df.loc[0,"rolling_fft_kurt_std"] = rolling_kurt_fft.std()

    df.loc[0,"rolling_fft_iqr_mean"] = q_75_fft.mean() - q_25_fft.mean()
    df.loc[0,"rolling_fft_iqr_std"] = q_75_fft.std() - q_25_fft.std()
    
    return df

In [6]:
%%time
data = pd.read_csv(train_data_path,dtype={'acoustic_data': np.int16, 'time_to_failure': np.float32})
                    #, engine='python',nrows = 50000)
data.rename({"acoustic_data":"signal","time_to_failure":"quaketime"},axis="columns",inplace=True)
data.head()

CPU times: user 2min 14s, sys: 23.6 s, total: 2min 38s
Wall time: 2min 22s


In [7]:
%%time
segment_size = 150_000
num_segments = int(np.round(len(data)/segment_size))
print(num_segments)
train = genTrainFeatures(data,num_segments,segment_size)
print(train.shape)

4194


/anaconda3/lib/python3.7/site-packages/pandas/core/window.py:222: ComplexWarning: Casting complex values to real discards the imaginary part
  values = _ensure_float64(values)



(4194, 49)
CPU times: user 1h 50min 12s, sys: 11.1 s, total: 1h 50min 23s
Wall time: 44min 20s


In [8]:
train.head()

,rolling_mean_10,rolling_std_10,rolling_fft_mean_10,rolling_fft_std_10,rolling_mean_50,rolling_std_50,rolling_fft_mean_50,rolling_fft_std_50,rolling_mean_100,rolling_std_100,...,rolling_fft_min_std,rolling_fft_max_mean,rolling_fft_max_std,rolling_fft_skew_mean,rolling_fft_skew_std,rolling_fft_kurt_mean,rolling_fft_kurt_std,rolling_fft_iqr_mean,rolling_fft_iqr_std,quaketime
0,4.884088,2.801810,7.562782,803.456076,4.883969,0.606041,7.059571,194.135166,4.883864,0.452295,...,2141.784331,2199.005729,2868.606813,-0.004251,0.343123,-0.012574,0.699152,1318.492262,-13.840733,1.430797
1,4.725732,3.924083,0.624315,441.654661,4.725729,0.764509,0.282004,125.462891,4.725623,0.496221,...,3013.650080,2513.878284,3539.965702,0.003050,0.317262,-0.083664,0.635381,1569.364520,13.473094,1.391499
2,4.906229,4.179743,0.534976,766.561886,4.906072,0.811312,0.166285,165.814530,4.906088,0.530153,...,3368.793378,2670.327076,3798.665651,0.003189,0.339385,-0.008290,0.673143,1599.942194,1.867970,1.353196
3,4.902291,4.299475,0.450443,505.896564,4.902059,0.959837,0.125840,144.223089,4.901946,0.581876,...,3128.307576,2598.300720,3712.924292,0.010637,0.319822,-0.039798,0.661223,1590.726806,26.787314,1.313798
4,4.908728,4.931307,7.633038,702.293151,4.908958,0.903961,7.256908,314.311143,4.909175,0.566268,...,3337.470420,2707.643176,3903.128672,0.007613,0.331600,-0.040144,0.678747,1670.763073,-7.951060,1.274400


In [9]:
prediction = ["quaketime"]
ignore_features = ['signal'] + prediction
features = [column for column in train.columns if column not in ignore_features]
(features,prediction)

(['rolling_mean_10',
  'rolling_std_10',
  'rolling_fft_mean_10',
  'rolling_fft_std_10',
  'rolling_mean_50',
  'rolling_std_50',
  'rolling_fft_mean_50',
  'rolling_fft_std_50',
  'rolling_mean_100',
  'rolling_std_100',
  'rolling_fft_mean_100',
  'rolling_fft_std_100',
  'rolling_mean_1000',
  'rolling_std_1000',
  'rolling_fft_mean_1000',
  'rolling_fft_std_1000',
  'rolling_q25_mean',
  'rolling_q25_std',
  'rolling_q75_mean',
  'rolling_q75_std',
  'rolling_q50_mean',
  'rolling_q50_std',
  'rolling_min_mean',
  'rolling_min_std',
  'rolling_max_mean',
  'rolling_max_std',
  'rolling_skew_mean',
  'rolling_skew_std',
  'rolling_kurt_mean',
  'rolling_kurt_std',
  'rolling_iqr_mean',
  'rolling_iqr_std',
  'rolling_fft_q25_mean',
  'rolling_fft_q25_std',
  'rolling_fft_q75_mean',
  'rolling_fft_q75_std',
  'rolling_fft_q50_mean',
  'rolling_fft_q50_std',
  'rolling_fft_min_mean',
  'rolling_fft_min_std',
  'rolling_fft_max_mean',
  'rolling_fft_max_std',
  'rolling_fft_skew_mean'

In [10]:
params = {'num_leaves': 50,
         'min_data_in_leaf': 10, 
         'objective':'regression',
         'max_depth': -1,
         'learning_rate': 0.001,
         "min_child_samples": 40,
         "boosting": "gbdt",
         "feature_fraction": 0.91,
         "bagging_freq": 1,
         "bagging_fraction": 0.91,
         "bagging_seed": 42,
         "metric": 'rmse',
         "lambda_l1": 0.1,
         "verbosity": -1,
         "nthread": 4,
         "random_state": 42}

In [11]:
X = train[features]
y = train[prediction]
scaler = StandardScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

In [12]:
(X_scaled.shape , y.shape)

((4194, 48), (4194, 1))

In [13]:
X_scaled.head()

,rolling_mean_10,rolling_std_10,rolling_fft_mean_10,rolling_fft_std_10,rolling_mean_50,rolling_std_50,rolling_fft_mean_50,rolling_fft_std_50,rolling_mean_100,rolling_std_100,...,rolling_fft_min_mean,rolling_fft_min_std,rolling_fft_max_mean,rolling_fft_max_std,rolling_fft_skew_mean,rolling_fft_skew_std,rolling_fft_kurt_mean,rolling_fft_kurt_std,rolling_fft_iqr_mean,rolling_fft_iqr_std
0,1.424050,-0.169471,1.141316,0.530983,1.423579,-0.132783,1.118857,-0.106051,1.423148,-0.098604,...,0.204397,-0.193227,-0.190787,-0.160019,-0.494178,1.467701,0.471366,1.303023,-0.164305,-0.371918
1,0.805579,0.005344,0.029601,-0.220336,0.805567,-0.033555,0.033091,-0.555918,0.805142,-0.046104,...,-0.002452,0.034720,0.013154,0.018816,0.360004,-1.973478,-1.617094,-1.704785,0.010625,0.344961
2,1.510521,0.045168,0.015286,0.454369,1.509902,-0.004248,0.014553,-0.291577,1.509946,-0.005550,...,-0.121654,0.127571,0.114484,0.087728,0.376265,0.970338,0.597209,0.076301,0.031946,0.040373
3,1.495141,0.063818,0.001742,-0.086931,1.494229,0.088754,0.008074,-0.433021,1.493770,0.056268,...,-0.054134,0.064697,0.067834,0.064888,1.247603,-1.632824,-0.328399,-0.485939,0.025520,0.694406
4,1.520281,0.162238,1.152573,0.320908,1.521172,0.053766,1.150470,0.681214,1.522001,0.037614,...,-0.124301,0.119381,0.138654,0.115554,0.893800,-0.065570,-0.338578,0.340628,0.081328,-0.217338


In [15]:
n_fold = 10
folds = KFold(n_splits=n_fold, shuffle=True, random_state=42)

In [16]:
%%time
for fold_n, (train_index, valid_index) in enumerate(folds.split(X_scaled)):
    
    train_act, train_valid = X_scaled.iloc[train_index], X_scaled.iloc[valid_index]
    
    train_x = train_act[features]
    print(train_x.shape)
    train_y = y.iloc[train_index]
    print(train_y.shape)
    
    train_x_valid = train_valid[features]
    train_y_valid = y.iloc[valid_index]
    
    model = lgb.LGBMRegressor(**params, n_estimators = 20000, n_jobs = -1)
    model.fit(train_x, train_y,eval_set=[(train_x, train_y), (train_x_valid, train_y_valid)],
              eval_metric='mae',verbose=1000, early_stopping_rounds=200)
    print(model.best_score_)

(3774, 48)
(3774, 1)
Training until validation scores don't improve for 200 rounds.
[1000]	training's rmse: 2.54641	training's l1: 2.05629	valid_1's rmse: 2.88436	valid_1's l1: 2.33201
[2000]	training's rmse: 2.14248	training's l1: 1.7153	valid_1's rmse: 2.77124	valid_1's l1: 2.191
[3000]	training's rmse: 1.90575	training's l1: 1.51856	valid_1's rmse: 2.75935	valid_1's l1: 2.15135
Early stopping, best iteration is:
[2898]	training's rmse: 1.92659	training's l1: 1.53572	valid_1's rmse: 2.75889	valid_1's l1: 2.15313
defaultdict(<class 'dict'>, {'training': {'rmse': 1.9265863380808503, 'l1': 1.5357217224031947}, 'valid_1': {'rmse': 2.7588850032559913, 'l1': 2.1531320571042305}})
(3774, 48)
(3774, 1)
Training until validation scores don't improve for 200 rounds.
[1000]	training's rmse: 2.54838	training's l1: 2.05577	valid_1's rmse: 2.79506	valid_1's l1: 2.29193
[2000]	training's rmse: 2.14334	training's l1: 1.71417	valid_1's rmse: 2.65035	valid_1's l1: 2.16007
[3000]	training's rmse: 1.905

In [17]:
model

LGBMRegressor(bagging_fraction=0.91, bagging_freq=1, bagging_seed=42,
       boosting='gbdt', boosting_type='gbdt', class_weight=None,
       colsample_bytree=1.0, feature_fraction=0.91,
       importance_type='split', lambda_l1=0.1, learning_rate=0.001,
       max_depth=-1, metric='rmse', min_child_samples=40,
       min_child_weight=0.001, min_data_in_leaf=10, min_split_gain=0.0,
       n_estimators=20000, n_jobs=-1, nthread=4, num_leaves=50,
       objective='regression', random_state=42, reg_alpha=0.0,
       reg_lambda=0.0, silent=True, subsample=1.0,
       subsample_for_bin=200000, subsample_freq=0, verbosity=-1)

In [18]:
curr_pred = model.predict(X_scaled)
print(curr_pred)

[3.83641005 5.24461419 3.93584232 ... 8.74568821 8.93083568 9.03947234]


In [19]:
curr_pred = np.array(curr_pred)

In [20]:
rmse = sqrt(mean_squared_error(y.values, curr_pred))
print(rmse)

2.023354778086353


In [21]:
submissions = pd.read_csv(submission_file_path)
# submissions.head()

In [25]:
%%time
predictions = []
for idx,row in tqdm(submissions.iterrows(), total=submissions.shape[0]):
    data_path = test_folder+row['seg_id']+".csv"
    print(data_path)
    test = pd.read_csv(data_path,dtype={'acoustic_data': np.int16})
    test.rename({"acoustic_data":"signal"},axis="columns",inplace=True)
    test = genTestFeatures(test.signal)
    test_scaled = pd.DataFrame(scaler.transform(test), columns=test.columns)
    curr_pred = model.predict(test,num_iteration=model.best_iteration_)
#     print(test_scaled)
    print(curr_pred)
    predictions.append(curr_pred[0])
#     if idx == 3:
#         break
    del test

./data/test/seg_00030f.csv


/anaconda3/lib/python3.7/site-packages/pandas/core/window.py:222: ComplexWarning: Casting complex values to real discards the imaginary part
  values = _ensure_float64(values)


[7.54166706]
./data/test/seg_0012b5.csv


/anaconda3/lib/python3.7/site-packages/pandas/core/window.py:222: ComplexWarning: Casting complex values to real discards the imaginary part
  values = _ensure_float64(values)


[7.26070245]
./data/test/seg_00184e.csv


...

[7.38782441]
./data/test/seg_ffbd6a.csv


/anaconda3/lib/python3.7/site-packages/pandas/core/window.py:222: ComplexWarning: Casting complex values to real discards the imaginary part
  values = _ensure_float64(values)


[7.37613181]
./data/test/seg_ffe7cc.csv


/anaconda3/lib/python3.7/site-packages/pandas/core/window.py:222: ComplexWarning: Casting complex values to real discards the imaginary part
  values = _ensure_float64(values)


[7.16455055]

CPU times: user 2h 22min 19s, sys: 42.9 s, total: 2h 23min 2s
Wall time: 39min 48s


In [26]:
submissions['time_to_failure'] = predictions
submissions.to_csv(submission_file_path,index=False)

In [27]:
predictions

[7.541667062543227,
 7.260702454562402,
 7.183554681997616,
 7.734828093005698,
 7.11924401911165,
 7.144793544340005,
 7.653628427349219,
 7.285111907015305,
 7.254201184895134,
 7.534223434020761,
 7.988474183718385,
 7.660237403504399,
 7.582465021902825,
 7.028111260732523,
 7.291159999965894,
 7.5521550043223025,
 7.293713536777617,
 6.803879380486664,
 7.967479474421516,
 7.5497841687571245,
 7.759320152905323,
 7.62829250831347,
 7.373324702375219,
 7.628117986662857,
 7.176081655194857,
 7.345321298506267,
 7.984991712477227,
 7.1304367907276776,
 7.2200087933598684,
 7.80510509391271,
 7.478244184484738,
 7.837530057049614,
 7.150082502887197,
 7.614291039787296,
 7.403729817017891,
 7.0779493612972075,
 7.394739225922849,
 7.5131319062468345,
 7.673877565439357,
 7.201324372881782,
 7.523236620060526,
 7.218938669970342,
 7.8434873730011025,
 7.711210054187191,
 7.215504912585969,
 7.271431718658735,
 7.465317573293784,
 7.10471893514289,
 7.342842669563111,
 7.60300610059370

In [31]:
data_list = []
power_spectrum = np.fft.fft(pd.Series(range(50000)))
power_spectrum = np.absolute(power_spectrum)
data_list.extend(list(power_spectrum[:10000][::25]))

In [35]:
power_spectrum.shape,len(data_list)

((50000,), 400)

In [51]:
abc = [x for x in range(10)] 
print(abc[::4])

[0, 4, 8]
